In [1]:
import sys

sys.path.append("../../../")
sys.path.append("../")
import csv
import random
import sys
from typing import List, Union

import cloudpickle
import clu
from clu import CLU
from pydantic import BaseModel, Field
from tqdm import tqdm

from brain.sdk import BrainClient

cloudpickle.register_pickle_by_value(clu)

In [2]:
brain_client = BrainClient("http://127.0.0.1:8000")

In [3]:
class GPQAQuestion(BaseModel):
    question: str
    options: List[str]
    correct_answer: str

class QA(BaseModel):
    explanation: str = Field(..., description="Explanation for the answer")
    answer: str = Field(
        ...,
        description="Final answer (A/B/C/D/Uncertain) without any additional explanation.",
    )



class GPQADataset:
    def __init__(self, file_path: str):
        self.questions = self._load_questions(file_path)

    def _load_questions(self, file_path: str) -> List[GPQAQuestion]:
        questions = []
        with open(file_path, "r", encoding="utf-8") as file:
            reader = csv.DictReader(file)
            for row in reader:
                options = [
                    row["Correct Answer"],
                    row["Incorrect Answer 1"],
                    row["Incorrect Answer 2"],
                    row["Incorrect Answer 3"],
                ]
                question = GPQAQuestion(
                    question=row["Question"],
                    options=options,
                    correct_answer=row["Correct Answer"],
                )
                questions.append(question)
        return questions

    def __len__(self) -> int:
        return len(self.questions)

    def __getitem__(self, index: int) -> GPQAQuestion:
        return self.questions[index]

# Helper Functions
def shuffle_options_and_set_answer(question_data: GPQAQuestion):
    """Randomize the options and determine the new correct answer label."""
    options = question_data.options.copy()
    correct_answer = question_data.correct_answer

    # Shuffle the options
    random.shuffle(options)

    # Determine the new label for the correct answer
    correct_label = chr(options.index(correct_answer) + ord("A"))

    return options, correct_label


def format_options(options: List[str]) -> str:
    """Format the options into labeled form (A. option, B. option, etc.)"""
    labeled_options = [f"{chr(65 + i)}. {option}" for i, option in enumerate(options)]
    return "\n".join(labeled_options)


In [4]:
main_goal = """
Your role is to engage in meta-learning to master the process of analytical reasoning for solving complex scientific tasks. Your objective is not to memorize specific facts or solutions, but to develop a robust skill set for systematic thinking, logical inference, and adaptive problem-solving. A critical part of this process is identifying and learning from mistakes, not just specific errors but generalizable patterns of reasoning missteps and strategies for improvement. The core of your approach should focus on:

1. **Learning Analytical Reasoning**:
   Study the provided examples of scientific reasoning, including questions, reasoning processes, and solutions, to internalize the principles of logical analysis, structured problem-solving, and evidence-based decision-making. Identify common reasoning steps that lead to success or failure.

2. **Identifying Patterns and Strategies**:
   Recognize recurring patterns in logical structures, reasoning techniques, and scientific principles. Similarly, identify generalizable reasoning errors or flaws. Generalize strategies to address new and unseen questions while avoiding repeated mistakes.

3. **Meta-Cognition**:
   Reflect on your reasoning processes to understand how effective solutions are constructed and identify areas where reasoning strategies fail or lead to suboptimal outcomes. Develop insights into how to refine these strategies for flexible adaptation across diverse scientific domains.

4. **Building a Reasoning Skill Set**:
   Practice adaptive reasoning by applying learned patterns and strategies to new problems. Analyze tasks for unique features, avoiding reliance on memorized answers or specific facts. Note down general mistakes, such as over-reliance on assumptions, logical fallacies, or missed contextual clues, and integrate strategies to prevent them.

5. **Learning from Mistakes**:
   Continuously refine your reasoning by comparing your thought processes and solutions to provided answers. Explicitly document reasoning errors in a general format, such as flawed assumptions, incorrect prioritization of information, or overgeneralization. Use these insights to improve your problem-solving framework for future tasks.

6. **Focusing on Principles and Errors Over Facts**:
   Prioritize understanding and applying underlying scientific principles and logical methods over memorizing domain-specific details. At the same time, focus on avoiding generalized reasoning errors by learning from past mistakes. Ensure your answers are grounded in analytical thinking and sound reasoning.

Important: Do not memorize the specific answers or facts. Instead, focus on identifying general reasoning mistakes, noting these errors, and updating your approach to avoid them. Build a repository of effective reasoning strategies and error-prevention techniques.

By mastering this approach, you will develop the ability to tackle a wide range of scientific tasks effectively, avoiding repeated errors, and adapting to new challenges with confidence and precision.
"""

clu = CLU(brain_client=brain_client,main_goal=main_goal, memory_file="gpqa_dynamic_memory.json")

    # Load the dataset
dataset = GPQADataset(file_path="./gpqa_diamond.csv")

In [5]:
for _ in range(20):
    question_data = random.choice(dataset.questions)

    # Create the task and determine the correct answer
    shuffled_options, correct_label = shuffle_options_and_set_answer(question_data)
    task = f"Question: {question_data.question}\n"
    task += f"Options:\n{format_options(shuffled_options)}\n\n"
    task += "Pick the correct option after thinking step by step."

    # Expected output
    expected_output = QA(
        explanation=f"The correct answer is {correct_label} because of {question_data.correct_answer}.",
        answer=correct_label,
    )

    # Train using CLU
    # print(f"Iteration {iteration + 1}:")
    # print(f"Query: {task}")

    result = clu.process_query(input_query=task, correct_answer=expected_output)

    # Print results
    print(f"Expected Output: {expected_output.answer}\n")
    print("Inference Result:", result["inference_result"].answer)
    # Uncomment if you want to see detailed feedback
    # print("Feedback:", result["feedback"])
    print("\n")

Expected Output: D

Inference Result: D. λ2 < λ1


Expected Output: B

Inference Result: A. -16.0 kJ


Expected Output: D

Inference Result: B. 9.08 Angstrom


Expected Output: A

Inference Result: B


Expected Output: B

Inference Result: C


Expected Output: B

Inference Result: C. protein degradation and loss-of-function of the wild-type allele


Expected Output: D

Inference Result: A


Expected Output: A

Inference Result: A, B, C


Expected Output: C

Inference Result: C. \left\langle H\right\rangle \ll \Delta E


Expected Output: A

Inference Result: C. A 6H singlet at ~1 ppm and a 1H doublet at ~1.5 ppm


Expected Output: D

Inference Result: C2


Expected Output: D

Inference Result: C. 7.2 MeV, 32.8 MeV


Expected Output: A

Inference Result: A. 6.24x10^-7 M


Expected Output: C

Inference Result: C


Expected Output: D

Inference Result: A


Expected Output: B

Inference Result: C. green signal colocalizes with the red signal


Expected Output: C

Inference Result: C


Expec

In [13]:
result = clu.process_query(input_query="Learn how to think step by step on your own for complex reasoning problems. these will be added to context of LLM prompt for us to enable it to think step by step", correct_answer="<various techiniques for LLMs to improve reasoning skills>")
print(result)

{'inference_result': InferenceResult(answer='To develop systematic analytical reasoning skills for complex scientific tasks, implement the following structured approach: 1. **Reflective Practice**: Regularly assess your reasoning strategies by employing a systematic reflective practice framework (Knowledge Item 92), which helps identify assumptions and learning points. 2. **Problem Breakdown**: Use a methodical process to deconstruct complex problems into key components (Knowledge Item 88), establishing their relationships and mapping them visually. 3. **Adaptive Questioning**: Engage in active questioning techniques (Knowledge Item 94) to clarify the problem, improve clarity, and foster creativity in solution approaches. 4. **Error Identification**: Create a checklist of common reasoning errors (Knowledge Item 101) including assumptions that may lead to fallacies, and consciously review them as you progress through analysis. 5. **Meta-Cognitive Evaluation**: Conduct metacognitive anal